In [650]:
#dependencies
import numpy as np
import pandas as pd
import os
import csv

In [651]:
# We Import NBA PLayer's Career Profile

players = './resources/Players.csv'
career= pd.read_csv(players)

print('Number of Rows', len(career))
# career.count()

Number of Rows 3922


In [652]:
#  We Import NBA Player's Season Profiles

player_data = './resources/player_data.csv'
plyr_season = pd.read_csv(player_data)

# Renames the Column 'name' to 'Player'
plyr_season = plyr_season.rename(columns={'name': 'Player'})

print('Number of Rows', len(plyr_season))
# plyr_season.count()

Number of Rows 4550


In [653]:
# Merge the plyr_season & career Dataframes on 'Players'

Player_Profile_Dirty = pd.merge(plyr_season, career, how='left', on=["Player","Player"])

print('Number of Rows', len(Player_Profile_Dirty))
# Player_Profile_Dirty.count()

Number of Rows 4550


In [654]:
# Create DataFrames and edit Column Names

htwt = [
    'Player',
    'height_y',
    'weight_x']
player_htwt= Player_Profile_Dirty[htwt]

# Renames Columns
P_htwt = player_htwt.rename(columns={
    'height_y':'Height (cm)',
    'weight_x':'Weight (lbs)'})
print('length of P_htwt', len(P_htwt))


# -----------
college = [
    'Player',
    'college']
player_college= Player_Profile_Dirty[college]

# Renames Columns
P_college = player_college.rename(columns={
   'college':'College'})
print('length of P_college', len(P_college))


# -----------
new_columns = [
'Player',
'birth_state']
PP= Player_Profile_Dirty[new_columns]

# Renames Columns
PPD = PP.rename(columns={
    'birth_state':'State',})
print('length of Player_htwt', len(PPD))

# P_htwt
# P_college.count()
# PPD.count()


length of P_htwt 4550
length of P_college 4550
length of Player_htwt 4550


In [655]:
# We Import United States latitude and Longitude

LL= './resources/50_SLat_Lng.csv'
ltlg= pd.read_csv(LL)

# Renames Colums
lat_lng= ltlg.rename(columns={
    'Place Name':'State'})
# lat_lng

In [656]:
#  Merge Player Profile Dataframe  with Lat. & Lng Dataframe

Player_Profile_State = pd.merge(PPD,lat_lng, how='left', on=["State","State"])
print(len(Player_Profile_State))
Player_Profile_State.count()

4550


Player       4550
State        3353
Latitude     2928
Longitude    2928
dtype: int64

In [657]:
# Remove Players not Born in the USA (1622)
# Remove Duplicate Players by -> name and state

Player_State = Player_Profile_State.dropna(axis = 0, how ='any') 

print("Old data frame length:", len(Player_Profile_State),
      "\nNew data frame length:", len(Player_State),
      "\nNumber of Player Not Born In USA: ",(len(Player_Profile_State)-len(Player_State)))
# Player_State

Old data frame length: 4550 
New data frame length: 2928 
Number of Player Not Born In USA:  1622


In [693]:
# ADD USA BORN PLAYERS  College Height and Weight
# Remove Duplicate Players by -> name and state
# Remove player with imcomplete Ht & Wt Data
# Merge The Dataframes

Player_htwt = pd.merge(Player_State, P_htwt, how='left', on=["Player","Player"])
Player_htwt.drop_duplicates(subset =('Player', 'State'), keep = False, inplace = True)
P_htwt_Clean = Player_htwt

# .dropna(axis = 0, how ='any') 

Player_Profile = P_htwt_Clean
#                           , P_college, how='left', on=["Player","Player"])
# Player_Profile = pd.merge(Player_htwt, P_college, how='left', on=["Player","Player"])

print("Old data frame length:", len(Player_State),
      "\nNew data frame length:", len(Player_Profile),
      "\nNumber of Players with Incomplete Ht&Wt Data: ",len(Player_State)-len(Player_Profile))
Player_htwt.count()
Player_Profile.count()

Old data frame length: 2928 
New data frame length: 2845 
Number of Players with Incomplete Ht&Wt Data:  83


Player          2845
State           2845
Latitude        2845
Longitude       2845
Height (cm)     2845
Weight (lbs)    2844
dtype: int64

In [694]:
import gmaps
from api_keys import g_key

In [695]:
gmaps.configure(api_key=g_key)


In [696]:
Player_Profile['MapPt'] = pd.Series(3, index=Player_Profile.index)

locations = Player_Profile[["Latitude", "Longitude"]]
birth_state = Player_Profile["MapPt"].astype(float)
Player_Profile.count()

Player          2845
State           2845
Latitude        2845
Longitude       2845
Height (cm)     2845
Weight (lbs)    2844
MapPt           2845
dtype: int64

In [697]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=birth_state, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
# print(Player_Profile['State'].value_counts())
fig

Figure(layout=FigureLayout(height='420px'))

In [698]:
# We Import NBA Teams Location

tll = './resources/NBATeamLL.csv'
Team_LL= pd.read_csv(tll)
# Team_LL

In [699]:
# # We Import NBA Player Season Stats

Season_stats = './resources/Seasons_Stats.csv'
stats_season = pd.read_csv(Season_stats)
#  Renames Columns

dirty_stats = stats_season.rename(columns={
    'Tm': 'Team',
    'Pos':'Position (Season)',
    'TRB':'Rebounds',
    'STL':'Steals',
    'BLK':'Blocks',
    'AST':'Assists',
    'PTS':'Points',
    'PER':'Player Rating'})
# dirty_stats

In [700]:
season_stats = pd.merge(dirty_stats, Team_LL, how='left', on=["Team","Team"])
season_stats.count()

Unnamed: 0           24691
Year                 24624
Player               24624
Position (Season)    24624
Age                  24616
Team                 24624
G                    24624
GS                   18233
MP                   24138
Player Rating        24101
TS%                  24538
3PAr                 18839
FTr                  24525
ORB%                 20792
DRB%                 20792
TRB%                 21571
AST%                 22555
STL%                 20792
BLK%                 20792
TOV%                 19582
USG%                 19640
blanl                    0
OWS                  24585
DWS                  24585
WS                   24585
WS/48                24101
blank2                   0
OBPM                 20797
DBPM                 20797
BPM                  20797
VORP                 20797
FG                   24624
FGA                  24624
FG%                  24525
3P                   18927
3PA                  18927
3P%                  15416
2

In [701]:
# Select only the Columns that are needed.

new_columns = [
    'Year',
    'Player',
    'Position (Season)',
    'Team',
    'Team Name',
    'Team Lat',
    'Team Lng',
    'Rebounds',
    'Steals',
    'Blocks',
    'Assists',
    'Points',
    'Player Rating']
stats_col= season_stats[new_columns]
# stats_col

In [708]:
# Select only the 2010-2015 Seaason statistic

STATS = stats_col.loc[(stats_col['Year']>=2010)&(stats_col['Year']<2015)]
STATS.count()

Year                 2938
Player               2938
Position (Season)    2938
Team                 2938
Team Name            2938
Team Lat             2938
Team Lng             2938
Rebounds             2938
Steals               2938
Blocks               2938
Assists              2938
Points               2938
Player Rating        2936
dtype: int64

In [711]:
# Merge the Clean PLayer Profile with the Clean Player Stats

ZZZ = pd.merge(STATS,Player_Profile, how='left', on=["Player","Player"])
ZZZ['MapPt'] = pd.Series(3, index=ZZZ.index)

NBA_Clean = ZZZ.dropna(axis = 0, how ='any') 
NBA_Clean.count()

Year                 2216
Player               2216
Position (Season)    2216
Team                 2216
Team Name            2216
Team Lat             2216
Team Lng             2216
Rebounds             2216
Steals               2216
Blocks               2216
Assists              2216
Points               2216
Player Rating        2216
State                2216
Latitude             2216
Longitude            2216
Height (cm)          2216
Weight (lbs)         2216
MapPt                2216
dtype: int64

In [715]:
Profile = pd.merge(NBA_Clean, P_college, how='left', on=["Player","Player"])
Profile.count()

Year                 2216
Player               2216
Position (Season)    2216
Team                 2216
Team Name            2216
Team Lat             2216
Team Lng             2216
Rebounds             2216
Steals               2216
Blocks               2216
Assists              2216
Points               2216
Player Rating        2216
State                2216
Latitude             2216
Longitude            2216
Height (cm)          2216
Weight (lbs)         2216
MapPt                2216
College              2044
dtype: int64

In [704]:

Clean_data = ZZZ.dropna(axis = 0, how ='any') 

print("Old data frame length:", len(ZZZ), "\nNew data frame length:",  
       len(Clean_data), "\nNumber of rows with at least 1 NA value: ", 
       (len(ZZZ)-len(Clean_data)))

Old data frame length: 2938 
New data frame length: 2216 
Number of rows with at least 1 NA value:  722


In [705]:
# Player_Profile_Clean['college'].value_counts()